<a href="https://colab.research.google.com/github/zyf2021/-/blob/main/Detectron_Wine_v_0_0_1_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Эта версия обучает модель и после обучения генерирует видео с масками. Можно сдавать

МИНУСЫ
- [ ] Видео генерируется долго.
- [ ] В режиме реального времени смотреть нельзя.
- [ ] Видео весит очень много
- [ ] Работа с кодеками и разрешением

ПЛЮСЫ
- [x] ОНО РАБОТАЕТ



<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">



## Перед началом

In [ ]:
!nvidia-smi

Sat Mar  9 12:03:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 ...    Off | 00000000:0A:00.0  On |                  N/A |
| 26%   28C    P8               9W / 100W |    401MiB /  4096MiB |     34%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install Detectron2 and dependencies

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'

  Using cached PyYAML-5.1-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
dask 2023.6.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
roboflow 1.1.21 requires PyYAML>=5.3.1, but you have pyyaml 5.1 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 5.1 which is incompatible.
jupyter-events 0.6.3 requires pyyaml>=5.3, but you have pyyaml 5.1 which is incompatible.
distributed 2023.6.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
superv

In [ ]:
dist = distutils.core.run_setup("./detectron2/setup.py")

In [ ]:
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}

In [ ]:
sys.path.insert(0, os.path.abspath('./detectron2'))

Проверим версии


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/bin/bash: строка 1: nvcc: команда не найдена
torch:  2.2 ; cuda:  cu118
detectron2: 0.6


In [ ]:
# COMMON LIBRARIES
import os
import cv2

from datetime import datetime
#from google.colab.patches import cv2_imshow

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

import numpy as np
import cv2 as cv


# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

import tqdm


matplotlib data path: /home/ary/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data
CONFIGDIR=/home/ary/.config/matplotlib
interactive is False
platform is linux
CACHEDIR=/home/ary/.cache/matplotlib
Using fontManager instance from /home/ary/.cache/matplotlib/fontlist-v330.json


## COCO Format Dataset

### Download

Нужен формат `COCO Segmentation`

Структура выглядить следующим образом:

```
dataset-directory/
├─ README.dataset.txt
├─ README.roboflow.txt
├─ train
│  ├─ train-image-1.jpg
│  ├─ train-image-1.jpg
│  ├─ ...
│  └─ _annotations.coco.json
├─ test
│  ├─ test-image-1.jpg
│  ├─ test-image-1.jpg
│  ├─ ...
│  └─ _annotations.coco.json
└─ valid
   ├─ valid-image-1.jpg
   ├─ valid-image-1.jpg
   ├─ ...
   └─ _annotations.coco.json
```

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Xzr1zJGyLIIlrEkPj2jz")
project = rf.workspace("zyf-ylbpa").project("my_dataset-ppyv8")
version = project.version(2)
dataset = version.download("coco-segmentation")

  Obtaining dependency information for PyYAML>=5.3.1 from https://files.pythonhosted.org/packages/7b/5e/efd033ab7199a0b2044dab3b9f7a4f6670e6a52c089de572e928d2873b06/PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (757 kB)
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.1
    Uninstalling PyYAML-5.1:
      Successfully uninstalled PyYAML-5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
Starting new HTTPS connection (1): api.roboflow.com:443
https://api.r

### Register

When you use Detectron2, before you actually train the model you need to [register it](https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#register-a-coco-format-dataset).

In [ ]:
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

We can now confirm that our custom dataset was correctly registered using [MetadataCatalog](https://detectron2.readthedocs.io/en/latest/modules/data.html#detectron2.data.MetadataCatalog).

In [ ]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['my_dataset-train', 'my_dataset-test', 'my_dataset-valid']

### Visualize

Let's take a look at single entry from out train dataset.

In [ ]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
metadata

namespace(name='my_dataset-train',
          json_file='/home/ary/Документы/my_dataset-2/train/_annotations.coco.json',
          image_root='/home/ary/Документы/my_dataset-2/train',
          evaluator_type='coco',
          thing_classes=['country-road', 'post', 'wine'],
          thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2})

In [ ]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata,
    scale=0.8,
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
#cv2.imshow("", out.get_image()[:, :, ::-1])
#k = cv2.waitKey(33)
#if k==27:    # Esc key to stop
#        break


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loaded 505 images in COCO format from /home/ary/Документы/my_dataset-2/train/_annotations.coco.json
findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
findfont: score(FontEntry(fname='/home/ary/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUni.ttf', name='STIXNonUnicode', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
findfont: score(FontEntry(fname='/home/ary/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizThreeSymReg.ttf', name='STIXSizeThreeSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
findfont: score(FontEntry(fname='/home/ary/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono-BoldOblique.ttf', name='DejaVu Sans Mono', style='oblique', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335
findfont: score(FontEntry(fname='/home/ary/an

## Train Model Using Custom COCO Format Dataset

### Configuration

In [ ]:
# HYPERPARAMETERS
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x"
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
MAX_ITER = 500
EVAL_PERIOD = 200
BASE_LR = 0.001
NUM_CLASSES = 3

# OUTPUT DIR
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): Bottlene


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loaded 505 images in COCO format from /home/ary/Документы/my_dataset-2/train/_annotations.coco.json
Removed 3 images with no usable annotations. 502 images left.
Distribution of instances among all 3 categories:
|   category   | #instances   |  category  | #instances   |  category  | #instances   |
|:------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| country-road | 0            |    post    | 177          |    wine    | 412          |
|              |              |            |              |            |              |
|    total     | 589          |            |              |            |              |
[DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
Using training sampler TrainingSampler
Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Serializing 502 elements to byte tensors and concaten

SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. These values will be ignored.


[DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl ...
URL https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl cached in /home/ary/.torch/iopath_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl
[Checkpointer] Loading from /home/ary/.torch/iopath_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl ...
Reading a file from 'Detectron2 Model Zoo'


...

No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


 eta: 0:01:07  iter: 399  total_loss: 0.656  loss_cls: 0.09644  loss_box_reg: 0.3942  loss_mask: 0.1216  loss_rpn_cls: 0.002116  loss_rpn_loc: 0.01448    time: 0.6689  last_time: 0.7826  data_time: 0.0024  last_data_time: 0.0023   lr: 0.0007982  max_mem: 2417M
 eta: 0:00:53  iter: 419  total_loss: 0.5479  loss_cls: 0.078  loss_box_reg: 0.3588  loss_mask: 0.1041  loss_rpn_cls: 0.001529  loss_rpn_loc: 0.006445    time: 0.6692  last_time: 0.5579  data_time: 0.0025  last_data_time: 0.0023   lr: 0.00083816  max_mem: 2417M
 eta: 0:00:40  iter: 439  total_loss: 0.6469  loss_cls: 0.1382  loss_box_reg: 0.3403  loss_mask: 0.1152  loss_rpn_cls: 0.002968  loss_rpn_loc: 0.007877    time: 0.6682  last_time: 0.6348  data_time: 0.0025  last_data_time: 0.0025   lr: 0.00087812  max_mem: 2417M
 eta: 0:00:26  iter: 459  total_loss: 0.7342  loss_cls: 0.1348  loss_box_reg: 0.4199  loss_mask: 0.1338  loss_rpn_cls: 0.002525  loss_rpn_loc: 0.008971    time: 0.6698  last_time: 0.7755  data_time: 0.0024  last_da


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loaded 23 images in COCO format from /home/ary/Документы/my_dataset-2/test/_annotations.coco.json
[DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Serializing 23 elements to byte tensors and concatenating them all ...
Serialized dataset takes 0.03 MiB


No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

[DetectionCheckpointer] Loading from my_dataset/mask_rcnn_R_101_FPN_3x/2024-03-09-12-04-03/model_final.pth ...
[Checkpointer] Loading from my_dataset/mask_rcnn_R_101_FPN_3x/2024-03-09-12-04-03/model_final.pth ...


In [ ]:
dataset_valid = DatasetCatalog.get(TEST_DATA_SET_NAME) # VALID_DATA_SET_NAME

for d in dataset_valid:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    print(1)
    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=metadata,
        scale=0.8,
        instance_mode=ColorMode.SEGMENTATION
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow('', out.get_image()[:, :, ::-1])
    cv2.waitKey()

# Работа с видео

## Изменим формат видео

In [ ]:
codec, file_ext = ("mp4v", ".mp4")

In [ ]:
# Открыть видеофайл для чтения
input_video_path = '/home/ary/Документы/MVI_8609.MOV'
video_capture = cv2.VideoCapture(input_video_path)

# Создать объект для записи видео в формате MP4
output_video_path = '/home/ary/Документы/MVI_8609.mp4'
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*codec)
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Читать кадры из входного видео, изменять их формат и записывать в выходное видео
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break
    output_video.write(frame)

# Освободить ресурсы
video_capture.release()
output_video.release()

print("Видео успешно сконвертировано из MOV в MP4!")

Видео успешно сконвертировано из MOV в MP4!


## Загрузим видео для анализа

In [ ]:
codec, file_ext = ("mp4v", ".mp4")
video_input_path = '/home/ary/Документы/MVI_8610.mp4'
video_output_path = '/home/ary/Документы/MVI_8610_output.mp4' #output_fname = args.output

video_input = cv2.VideoCapture(video_input_path)
width = int(video_input.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_input.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_input.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*codec)

num_frames = int(video_input.get(cv2.CAP_PROP_FRAME_COUNT))
basename = os.path.basename(video_input_path)

video_output = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))


width, height, fps, num_frames, basename

(1280, 720, 50, 132, 'MVI_8610.mp4')

In [ ]:
metadata

namespace(name='my_dataset-train',
          json_file='/home/ary/Документы/my_dataset-2/train/_annotations.coco.json',
          image_root='/home/ary/Документы/my_dataset-2/train',
          evaluator_type='coco',
          thing_classes=['country-road', 'post', 'wine'],
          thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2})

In [ ]:
while True:
    ret, frame = video_input.read()
    if not ret:
        break
    outputs = predictor(frame)
    visualizer = Visualizer(frame[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.SEGMENTATION)
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    frame_to_write = out.get_image()[:, :, ::-1]
    print(torch.from_numpy(frame_to_write).shape)
    video_output.write(frame_to_write)

    cv2.imshow(basename, frame_to_write)
    if cv2.waitKey(1) == 27:
        break  # esc to quit


video_input.release()
video_output.release()
cv2.destroyAllWindows()

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

In [ ]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
while True:
    ret, frame = video.read()
    if not ret:
        break
    outputs = predictor(frame)
    visualizer = Visualizer(frame[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.SEGMENTATION)
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    frame_to_write = out.get_image()[:, :, ::-1]

    output_file.write(frame_to_write)
    cv2.imshow(basename, frame_to_write)
    if cv2.waitKey(1) == 27:
        break  # esc to quit
video.release()
output_file.release()
cv2.destroyAllWindows()

NameError: name 'video' is not defined